In [1]:
import pprint
import urllib.parse
import json5
from qwen_agent.agents import Assistant
from qwen_agent.tools.base import BaseTool, register_tool


# 步骤 1（可选）：添加一个名为 `my_image_gen` 的自定义工具。
@register_tool('my_image_gen')
class MyImageGen(BaseTool):
    # `description` 用于告诉智能体该工具的功能。
    description = 'AI 绘画（图像生成）服务，输入文本描述，返回基于文本信息绘制的图像 URL。'
    # `parameters` 告诉智能体该工具有哪些输入参数。
    parameters = [{
        'name': 'prompt',
        'type': 'string',
        'description': '期望的图像内容的详细描述',
        'required': True
    }]

    def call(self, params: str, **kwargs) -> str:
        # `params` 是由 LLM 智能体生成的参数。
        prompt = json5.loads(params)['prompt']
        prompt = urllib.parse.quote(prompt)
        return json5.dumps(
            {'image_url': f'https://image.pollinations.ai/prompt/{prompt}'},
            ensure_ascii=False)


# 步骤 2：配置您所使用的 LLM。
llm_cfg = {
    # 使用 DashScope 提供的模型服务：
    'model': 'qwen-max',
    'model_server': 'dashscope',
    'api_key': 'sk-882e296067b744289acf27e6e20f3ec0',
    'generate_cfg': {
        'top_p': 0.8
    }
}

llm_cfg = {
    # 使用 DashScope 提供的模型服务：
    'model': 'deepseek-v3',
    'model_server': 'https://dashscope.aliyuncs.com/compatible-mode/v1',
    'api_key': 'sk-882e296067b744289acf27e6e20f3ec0',
    'generate_cfg': {
        'top_p': 0.8
    }
}

# 步骤 3：创建一个智能体。这里我们以 `Assistant` 智能体为例，它能够使用工具并读取文件。
system_instruction = '''你是一个乐于助人的AI助手。
在收到用户的请求后，你应该：
- 首先绘制一幅图像，得到图像的url，
- 然后运行代码`request.get`以下载该图像的url，
- 最后从给定的文档中选择一个图像操作进行图像处理。
用 `plt.show()` 展示图像。
你总是用中文回复用户。'''
tools = ['my_image_gen', 'code_interpreter']  # `code_interpreter` 是框架自带的工具，用于执行代码。
import os
# 获取文件夹下所有文件
file_dir = os.path.join('./', 'docs')
files = []
if os.path.exists(file_dir):
    # 遍历目录下的所有文件
    for file in os.listdir(file_dir):
        file_path = os.path.join(file_dir, file)
        if os.path.isfile(file_path):  # 确保是文件而不是目录
            files.append(file_path)
print('files=', files)

bot = Assistant(llm=llm_cfg,
                system_message=system_instruction,
                function_list=tools,
                files=files)

# 步骤 4：作为聊天机器人运行智能体。
messages = []  # 这里储存聊天历史。
query = "介绍下雇主责任险"
# 将用户请求添加到聊天历史。
messages.append({'role': 'user', 'content': query})
response = []
current_index = 0
for response in bot.run(messages=messages):
    if current_index == 0:
        # 尝试获取并打印召回的文档内容
        if hasattr(bot, 'retriever') and bot.retriever:
            print("\n===== 召回的文档内容 =====")
            retrieved_docs = bot.retriever.retrieve(query)
            if retrieved_docs:
                for i, doc in enumerate(retrieved_docs):
                    print(f"\n文档片段 {i+1}:")
                    print(f"内容: {doc.page_content}")
                    print(f"元数据: {doc.metadata}")
            else:
                print("没有召回任何文档内容")
            print("===========================\n")
        #break

    current_response = response[0]['content'][current_index:]
    current_index = len(response[0]['content'])
    print(current_response, end='')
# 将机器人的回应添加到聊天历史。
#messages.extend(response)

files= ['./平安团体险\\1-平安商业综合责任保险（亚马逊）.txt', './平安团体险\\2-雇主责任险.txt', './平安团体险\\3-平安企业团体综合意外险.txt', './平安团体险\\4-雇主安心保.txt', './平安团体险\\5-施工保.txt', './平安团体险\\6-财产一切险.txt', './平安团体险\\7-平安装修保.txt', './平安团体险\\平安产险交通出行意外伤害保险（互联网版）产品说明.pdf', './平安团体险\\平安产险交通工具意外伤害保险（互联网版）条款.pdf', './平安团体险\\平安企业团体综合意外险(互联网版)适用条款.pdf', './平安团体险\\平安商业综合责任保险（亚马逊）.pdf', './平安团体险\\平安境内紧急医疗救援服务条款.pdf', './平安团体险\\平安附加疾病身故保险条款.pdf']


2025-03-10 15:10:17,774 - memory.py - 113 - INFO - {"keywords_zh": ["雇主责任险", "雇主", "责任", "保险"], "keywords_en": ["employer liability insurance", "employer", "liability", "insurance"], "text": "介绍下雇主责任险"}
2025-03-10 15:10:18,402 - simple_doc_parser.py - 380 - INFO - Read parsed ./平安团体险\1-平安商业综合责任保险（亚马逊）.txt from cache.
2025-03-10 15:10:18,409 - doc_parser.py - 108 - INFO - Start chunking ./平安团体险\1-平安商业综合责任保险（亚马逊）.txt (1-平安商业综合责任保险（亚马逊）.txt)...
2025-03-10 15:10:18,414 - doc_parser.py - 126 - INFO - Finished chunking ./平安团体险\1-平安商业综合责任保险（亚马逊）.txt (1-平安商业综合责任保险（亚马逊）.txt). Time spent: 0.0 seconds.
2025-03-10 15:10:18,422 - simple_doc_parser.py - 380 - INFO - Read parsed ./平安团体险\2-雇主责任险.txt from cache.
2025-03-10 15:10:18,425 - doc_parser.py - 108 - INFO - Start chunking ./平安团体险\2-雇主责任险.txt (2-雇主责任险.txt)...
2025-03-10 15:10:18,429 - doc_parser.py - 126 - INFO - Finished chunking ./平安团体险\2-雇主责任险.txt (2-雇主责任险.txt). Time spent: 0.0 seconds.
2025-03-10 15:10:18,435 - simple_doc_parser.py - 380 - 

雇主责任险是一种为企业提供保障的保险产品，主要针对企业在雇佣员工过程中可能面临的法律责任风险。以下是雇主责任险的详细介绍：

### 1. **保障范围**
雇主责任险的保障范围包括：
- **死亡赔偿金**：员工因工作原因导致死亡，保险公司将赔付相应的死亡赔偿金。
- **伤残赔偿金**：员工因工作原因导致伤残，保险公司将根据伤残等级赔付相应的赔偿金。
- **医疗费用**：员工因工作原因受伤或患病，保险公司将赔付相应的医疗费用。
- **误工费用**：员工因工作原因受伤或患病导致无法工作，保险公司将赔付相应的误工费用。
- **法律诉讼费用**：员工因工作原因受伤或患病，企业因此面临法律诉讼，保险公司将赔付相应的法律诉讼费用。

### 2. **产品亮点**
- **雇主责任转嫁给保险公司**：企业通过购买雇主责任险，将员工在工作中可能面临的风险责任转嫁给保险公司，降低企业的用工风险。
- **降低工伤纠纷**：通过保险赔付，减少企业与员工之间的工伤纠纷，维护企业声誉。
- **提高员工保障福利**：为员工提供更全面的保障，提高员工的满意度和忠诚度。
- **税前列支**：企业购买雇主责任险的保费可以在企业所得税前列支，享受税收优惠。

### 3. **产品区别**
雇主责任险通常有多个版本，如A款、B款和94款，不同版本在保障范围和赔付标准上有所区别：
- **A款**：依据《工伤保险条例》第十四条、第十五条规定，员工因工作遭受意外事故或罹患职业病所致伤、残或死亡，保险公司履行赔偿责任。
- **B款**：从《工伤保险条例》中摘录部分工伤情形，员工因摘录情形中所致伤、残或死亡，保险公司履行赔偿责任。
- **94款**：员工因工作遭受意外事故或罹患职业病所致伤、残或死亡，依法保险公司履行赔偿责任，保费及赔偿限额根据雇员工资总额进行核算。

### 4. **理赔案例**
以下是一些雇主责任险的理赔案例：
- **案例一**：某连锁餐饮管理公司为1100余名员工购买了雇主责任险，每人赔偿限额30万元。2018年9月8日，雇员张某在上班途中发生交通意外死亡，保险公司赔付30万元。
- **案例二**：某客户投保雇主责任险，每人赔偿限额80万元。2019年5月15日，雇员任某在西藏林芝工作时突发胸痛死亡，保险公司赔付80万元。
- **案例三**：某制冷公司投保